# Preparing Data for São Paulo SVF calculation

In [17]:
import os
import rioxarray
from rioxarray import merge
import geopandas as gpd
from shapely.geometry import mapping
import numpy as np

In [2]:
gdf_sp = gpd.read_file('test/data/sao-paulo-test.gpkg')

In [3]:
mdt = 'data/raster/MDT_sampa-ZSTD.tif'
bhm = 'data/raster/0-BHM-sao-paulo-city-all.tif'
result = 'data/mds-for-svf-sao-paulo.tif'

In [4]:
gdf_sp.geometry.apply(mapping)

0    {'type': 'Polygon', 'coordinates': (((330049.8...
Name: geometry, dtype: object

In [5]:
xmdt = rioxarray.open_rasterio(mdt, chunks=True, masked=True).rio.clip(gdf_sp.geometry.apply(mapping), gdf_sp.crs, from_disk=True, all_touched=True)
xbhm = rioxarray.open_rasterio(bhm, chunks=True, masked=True).rio.clip(gdf_sp.geometry.apply(mapping), gdf_sp.crs, from_disk=True, all_touched=True)

In [24]:
xbhm

<xarray.DataArray (band: 1, y: 7482, x: 8623)>
array([[[ 3.73084998,  3.80812383,  3.8182807 , ..., 36.54111481,
         44.60840607, 62.94744492],
        [ 3.71425104,  3.72494745,  3.80740142, ..., 39.36030197,
         47.22966003, 66.40991974],
        [ 3.62820816,  3.69623327,  3.72102809, ..., 43.2565155 ,
         53.06160355, 66.36553192],
        ...,
        [ 6.90063477,  6.83301306,  6.79411077, ...,  6.708395  ,
          6.76536417,  6.78495646],
        [ 6.9648757 ,  6.91109276,  6.84579039, ...,  6.77257824,
          6.81838846,  6.88174295],
        [ 6.96978426,  6.9125104 ,  6.84079456, ...,  6.81587553,
          6.89173746,  6.9172473 ]]])
Coordinates:
  * x            (x) float64 3.3e+05 3.301e+05 3.301e+05 ... 3.344e+05 3.344e+05
  * y            (y) float64 7.394e+06 7.394e+06 7.394e+06 ... 7.39e+06 7.39e+06
  * band         (band) int64 1
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [26]:
xmds.shape, xmdt.shape, xbhm.shape

((1, 7483, 8624), (1, 7481, 8623), (1, 7482, 8623))

In [19]:
# xmdt.plot()

In [33]:
xmds = merge.merge_arrays([xmdt, xbhm], method='sum')

In [34]:
xmds.rio.to_raster('tmp/raster_mdt_bhm.tiff')